In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0)
g.add_node(1)
g.add_edge(0, 1, arrows='to')
g.show("test1.html")

In [3]:
from pyvis.network import Network

g = Network(width='90%', notebook=True)
g.add_node(0)
g.add_node(1)
g.add_edge(0, 1, arrows='to')
g.show_buttons(filter_=['physics'])

g.show("test2.html")

In [4]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0, label='A')
g.add_node(1, label='B')
g.add_edge(0, 1, arrows='to')
g.show("test3.html")

In [5]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0, label='A', title='I am A')
g.add_node(1, label='B', title='I am B')
g.add_node(2, label='C', title='new C')
g.add_edge(0, 1, arrows='to')
g.add_edge(0, 2, arrows='to')
g.show("test4.html")

In [6]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0, label='A', title='I am A', shape='diamond')
g.add_node(1, label='B', title='I am B', shape='box')
g.add_node(2, label='C', title='new C', shape='triangle', size=50)
g.add_node(3, label='D', shape='triangleDown')
g.add_node(4, label='E', shape='square', color='red')
g.add_node(5, label='F', shape='database', color='green', shadow=True)
g.add_edge(0, 1, arrows='to')
g.add_edge(0, 2, arrows='to')
g.show("test5.html")

In [7]:
from pyvis.network import Network
import networkx as nx

def getLocalName(node):
    if node.find('/') > 0 and node.find('#') > 0:
        return node[node.rindex('/')+1:] if node.rindex('/') > node.rindex('#') else node[node.rindex('#')+1:]
    elif node.find('/') > 0:
        return node[node.rindex('/')+1:]
    elif node.find('#') > 0:
        return node[node.rindex('/')+1:]
    else:
        return node


def make_short_name(name):
    if len(name) > 20 :
        name = name[0:20]
    return name

def show_graph(results, ignore_property):
    g = Network('800px', '1200px', notebook=True)
    node = {}
    groups = {}
    node_num = 0;
    group_num = 0;
    edge_list = []
    
    for result in results["results"]["bindings"]:
        if not result["p"]["value"] in ignore_property:
            # 존재하지 않는 경우 디폴트로 처리
            if 'slabel' not in result:
                result['slabel'] = {'type': 'literal', 'value': getLocalName(result['s']['value'])}
            if 'olabel' not in result:
                result['olabel'] = {'type': 'literal', 'value': getLocalName(result['o']['value'])}
            if 'stype' not in result:
                result['stype'] = {'type': 'literal', 'value': ''}
            if 'otype' not in result:
                result['otype'] = {'type': 'literal', 'value': ''}

            # 노드
            if result['s']['type'] == 'uri' and not result['s']['value'] in node: 
                node[result['s']['value']] = {'id':node_num}
                node_num += 1

            if result['o']['type'] == 'uri' and not result['o']['value'] in node:
                node[result['o']['value']] = {'id':node_num}
                node_num += 1

            if 'stype'  in result and not result['stype']['value'] in groups:
                groups[result['stype']['value']] = group_num
                group_num += 1
            if 'otype'  in result and not result['otype']['value'] in groups:
                groups[result['otype']['value']] = group_num
                group_num += 1

            node[result['s']['value']].update({'label':result['slabel']['value'], 'type':result['stype']['value']})
            if result['o']['type'] == 'uri':
                node[result['o']['value']].update({'label':result['olabel']['value'], 'type':result['otype']['value']})
                edge_list.append((result['s']['value'], result['o']['value'], result['p']['value']))

    for n in node:
        g.add_node(n, size=25, label=make_short_name(node[n]['label']), title=n, group=groups[node[n]['type']])
    for e in edge_list:
        g.add_edge(e[0], e[1], arrows='to', label=getLocalName(e[2]), title=e[2])
    
    g.set_edge_smooth('dynamic')
    g.show_buttons(filter_=['physics'])
    return g

## 클러스터 1개로 pyvis 테스트

In [28]:
import pandas as pd

In [29]:
df = pd.read_csv('폭발파열_동시단어출현_cluster0.csv', index_col=0)

In [30]:
df

,골절,공구,면부,물질,부탄가스,상태,소각,소각장,쓰레기,용기,우측,인화,제거,청소,추정,폭발,현장
골절,3,0,1,2,0,0,0,0,0,1,3,0,1,0,3,3,1
공구,0,6,2,1,0,1,0,0,0,0,0,1,8,0,0,4,0
면부,1,2,7,0,1,2,2,1,2,2,1,0,5,0,2,7,2
물질,2,1,0,23,1,0,3,2,4,3,2,2,10,6,0,2,1
부탄가스,0,0,1,1,3,0,4,3,6,2,0,1,0,2,1,2,0
상태,0,1,2,0,0,3,0,0,0,1,0,0,4,0,0,3,0
소각,0,0,2,3,4,0,11,6,13,4,0,2,0,3,2,4,1
소각장,0,0,1,2,3,0,6,6,9,3,0,2,0,2,1,1,0
쓰레기,0,0,2,4,6,0,13,9,22,5,0,3,0,5,2,5,1
용기,1,0,2,3,2,1,4,3,5,4,1,1,1,1,1,3,0


In [31]:
nodes = df.columns.to_list()
nodes = [node.strip() for node in nodes]
nodes

['골절',
 '공구',
 '면부',
 '물질',
 '부탄가스',
 '상태',
 '소각',
 '소각장',
 '쓰레기',
 '용기',
 '우측',
 '인화',
 '제거',
 '청소',
 '추정',
 '폭발',
 '현장']

In [32]:
def get_edges(node: str, weights: list, all_nodes: list, minium_weight: int):
    
    nodes = all_nodes.copy()
    
    # Remove target node
    nodes.remove(node)
    
    # Create a list of edges with weights
    edges = [(node, connection, weight) for connection, weight in zip(nodes, weights)]
    
    # Get only edges with weights greater than the minimum weight
    edges = [edge for edge in edges if edge[2] >= minium_weight]
    
    return edges

In [33]:
def draw_network(
    nodes: list,
    df: pd.DataFrame,
    minium_weight: int = 0,
    repulsion: int = 100,
    spring_length=200,
):
    
    net = Network("500px", "500px", notebook=True)
    net.add_nodes(nodes) 

    # add edges
    for node, weights in df.iterrows():
        edges = get_edges(node, weights, nodes, minium_weight)
        net.add_edges(edges)

    # change node distance and spring length
    net.repulsion(repulsion, spring_length=spring_length)
    return net

In [34]:
net = draw_network(nodes, df, minium_weight=0, repulsion=100, spring_length=500)
net.show("match.html")

In [40]:
net = draw_network(nodes, df, minium_weight=3, repulsion=100, spring_length=500)
net.show_buttons(filter_=['physics'])
net.show("match.html")

In [24]:
net.show_buttons(filter_=True)

In [26]:
net.show_buttons(filter_=['physics'])